In [2]:
# MAKING ASYNC MULTIPROCESS VERSION
import asyncio
from concurrent.futures import ThreadPoolExecutor

# from IPython import display

import time
from lib_xml_tree import *
from lib_connection import *
from lib_config_loader import load_config

import tensorflow as tf
import datetime
from utils import label_map_util

import json
from ifm3dpy.device import O3D
# import argparse
from pathlib import Path
import copy, socket

import cv2                #  assumed – for AprilTag detection
import numpy as np
from ifm3dpy.pcic import FrameGrabber, buffer_id
import subprocess, logging
from typing import Tuple


tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)
# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)





class CameraManager:
    def __init__(self, cfg):
        self.cfg = cfg        
        
        self.state = "free" # free / calibration / inference / loading_weights / loading parameters 
        self.model = None
        self.detect_fn = None
        self.mode = "inference"   # inference / calibration

        self.o3d = O3D(ip=self.cfg["SENSOR_IP"])
        self._config_path = None  # path to the settings file of current settings
        self._config = None       # current camera settings config, read from self._config_path  
        

        # prepare to work
        self.set_inference_settings()


        
        

    def get_state(self):
        return self.state

    def load_model(self):
        # Load saved model and build the detection function
        self.model = tf.saved_model.load(self.cfg['PATH_TO_SAVED_MODEL'])
        self.detect_fn = self.model.signatures['serving_default']

    def calib_procedure(self):
        try:            
            self.mode  = "calibration"
            self.set_calibration_settings()
            self.state = "busy"
            
            # calibration logic




            
            self.set_inference_settings()
        finally:                
            self.mode  = "inference"
            self.state = "free"

        return result_kuka_frame




# ---------- private helpers ----------
    @staticmethod
    def _load_config_file(path: Path) -> dict:
        """Load configuration from disk into a Python dict."""
        with open(path, "r", encoding="utf‑8") as f:
            return json.load(f)

    def _push_config(self) -> None:
        """Send the current `self._config` JSON to the camera."""
        self.o3d.from_json(self._config)

    def _pull_config(self) -> None:
        """Fetch the live configuration from the camera into memory (`self._config`)."""
        self._config = self.o3d.to_json()

    # ---------- public API ----------
    def get_config(self) -> dict:
        """Return a **deep copy** of the current config so the caller can’t modify it in place."""
        return copy.deepcopy(self._config)

    def save_config_to_file(self, path: str | Path | None = None) -> None:
        """Persist the working configuration to disk."""
        path = Path(path or self._config_path)
        with open(path, "w", encoding="utf‑8") as f:
            json.dump(self._config, f, indent=2)

    def update_config(self, patch: dict, push: bool = True) -> None:
        """
        Apply a *patch* to the working config and, optionally,
        push the result to the camera immediately.

        Parameters
        ----------
        patch : dict
            Keys/values to merge into the current configuration.
            Nested dictionaries are merged recursively.
        push : bool, default=True
            If *True*, call `_push_config()` after merging.
        """
        # Recursively merge nested dictionaries
        def merge(a: dict, b: dict) -> None:
            for k, v in b.items():
                if isinstance(v, dict) and isinstance(a.get(k), dict):
                    merge(a[k], v)
                else:
                    a[k] = v

        merge(self._config, patch)
        if push:
            self._push_config()

    def reload_from_device(self) -> None:
        """Sync the local copy if someone changed the camera settings directly on the device."""
        self._pull_config()

    def set_calibration_settings(self) -> None:
        """Load calibration JSON and push it to the camera."""
        self.state = 'changing settings to calibration'
        
        cfg_path = Path(self.cfg["config_path_calibration_setting"]).expanduser().resolve()
        assert cfg_path.is_file(), f"Calibration file not found: {cfg_path}"
    
        self._config_path = cfg_path                  
        self._config = self._load_config_file(cfg_path)
        self._push_config()

        wait_camera_online(self.cfg["SENSOR_IP"])  # await for camera alive again
        self.mode = "calibration"   # inference / calibration
        self.state = "free"
        print("Calibration settings applied.")
    
    def set_inference_settings(self) -> None:        
        """Load inference JSON and push it to the camera."""
        self.state = 'changing settings to inference'
        
        cfg_path = Path(self.cfg["config_path_work_inference_setting"]).expanduser().resolve()
        assert cfg_path.is_file(), f"Inference file not found: {cfg_path}"
    
        self._config_path = cfg_path                   
        self._config = self._load_config_file(cfg_path)
        self._push_config()

        wait_camera_online(self.cfg["SENSOR_IP"])  # await for camera alive again
        self.mode = "inference"   # inference / calibration
        self.state = "free"
        print("Inference settings applied.")

    @staticmethod
    def wait_camera_online(ip: str, *, timeout: float = 10.0, interval: float = 0.5) -> None:
        """
        Blocking subproced while camera is rebooting
        await for camera ping feedback of timeout
        Makes RuntimeError, if no response from camera within timeout .
        """
        t0 = time.time()
        while True:
            #   -c 1  : послать 1 пакет
            #   -W 1  : ждать ответа 1 секунду
            if subprocess.call(["ping", "-c", "1", "-W", "1", ip],
                               stdout=subprocess.DEVNULL,
                               stderr=subprocess.DEVNULL) == 0:
                logging.info("Camera %s is back online (%.1f s)", ip, time.time() - t0)
                return
    
            if time.time() - t0 > timeout:
                raise RuntimeError(f"Camera {ip} did not come online in {timeout} s")
    
            time.sleep(interval)

    async def capture_apriltag_transform(
        self,
        *,
        save_raw: bool = False,
        root_dir: Path = Path(self.cfg["root_dir"]),
        timeout_ms: int = 100,
    ) -> Tuple[np.ndarray, float]:
        """
        Grab one frame by SW‑trigger, find an AprilTag, and return the 4×4
        homogeneous transform (camera → tag).
    
        Parameters
        ----------
        save_raw : bool, default=False
            If *True* the amplitude / distance / confidence / XYZ buffers are
            saved on disk under `root_dir/YYYY_MM_DD/{AM|PM}/{index}/`.
        root_dir : Path
            Top‑level folder for raw dumps (ignored when `save_raw` == False).
        timeout_ms : int
            `FrameGrabber.wait_for_frame()` timeout.
    
        Returns
        -------
        transform : np.ndarray  [4, 4]
        timestamp : float        Unix epoch (s)
    
        Raises
        ------
        RuntimeError
            When the camera is unreachable or no AprilTag is detected.
        """
    
        loop = asyncio.get_running_loop()
    
        # -----------------------------------------------------------------
        # 1. Prepare helpers that must NOT block the asyncio loop
        # -----------------------------------------------------------------
        def _ping(ip: str, tries: int = 1) -> bool:
            """True if the device replies to ICMP."""
            cmd = ["ping", "-c", str(tries), "-W", "1", ip]
            return subprocess.call(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL) == 0
    
        def _open_fg() -> FrameGrabber:
            """Create and start a new frame grabber."""
            cam = self.o3d                                 # already opened in __init__
            fg_ = FrameGrabber(cam, pcic_port=self.cfg["xmlrpc_port"])
            fg_.start([
                buffer_id.AMPLITUDE_IMAGE,
                buffer_id.RADIAL_DISTANCE_IMAGE,
                buffer_id.CONFIDENCE_IMAGE,
                buffer_id.XYZ,
            ])
            return fg_
    
        def _normalize_amplitude(buf: np.ndarray) -> np.ndarray:
            """Stretch uint16 → uint8 for debugging / storage."""
            arr = buf.copy()
            arr[arr >= 16200] = 0          # device‑specific cut‑off
            arr[:, 215:] = 0              # mask right margin
            mn, mx = arr.min(), arr.max()
            return ((arr - mn) * 255 / max(mx - mn, 1)).astype(np.uint8)
    
        def _detect(buf_amp: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
            """
            AprilTag detection stub.
            Returns (corner_px[4,2], transform_cam_tag[4,4]).
            Replace this with your real detector.
            """
            # Dummy detector: always fails
            raise RuntimeError("No AprilTag detected")
    
        # -----------------------------------------------------------------
        # 2. (Re)open the frame grabber – done in a thread
        # -----------------------------------------------------------------
        fg = await loop.run_in_executor(None, _open_fg)
    
        # -----------------------------------------------------------------
        # 3. Try to trigger & grab a frame (with basic retry)
        # -----------------------------------------------------------------
        for attempt in range(2):          # two shots: first may time out
            try:
                fg.sw_trigger()
                ok, frame = fg.wait_for_frame().wait_for(timeout_ms)
            except Exception as exc:
                logging.warning("Frame grabber error: %s", exc)
                ok = False
    
            if ok:
                break
    
            # Frame failed – check if the device is alive
            ip = self.cfg["SENSOR_IP"]
            alive = await loop.run_in_executor(None, _ping, ip)
            if not alive:
                raise RuntimeError(f"Camera {ip} is unreachable (ping failed)")
            await asyncio.sleep(0.1)      # give the sensor a moment
    
        if not ok:
            raise RuntimeError("Timeout while waiting for a frame")
    
        ts_epoch = frame.timestamps().image_time_ns / 1e9
    
        # -----------------------------------------------------------------
        # 4. Process the buffers (runs in executor to keep event‑loop free)
        # -----------------------------------------------------------------
        def _post_process():
            buf_amp  = frame.get_buffer(buffer_id.AMPLITUDE_IMAGE)
            buf_dist = frame.get_buffer(buffer_id.RADIAL_DISTANCE_IMAGE)
            buf_conf = frame.get_buffer(buffer_id.CONFIDENCE_IMAGE)
            buf_xyz  = frame.get_buffer(buffer_id.XYZ)          # shape (H,W,3)
    
            amp_u8 = _normalize_amplitude(buf_amp)
    
            corners_px, T_cam_tag = _detect(amp_u8)
    
            if save_raw:
                # Build folder  …/YYYY_MM_DD/AM|PM/N/
                now       = datetime.datetime.now()
                day_dir   = root_dir / now.strftime("%Y_%m_%d") / now.strftime("%p")
                index_dir = max([int(p.name) for p in day_dir.iterdir() if p.is_dir()] + [0]) + 1
                dump_dir  = day_dir / f"{index_dir:04d}"
                dump_dir.mkdir(parents=True, exist_ok=True)
    
                base = dump_dir / now.strftime("%Y_%m_%d_%H_%M_%S")
                np.save(base.with_suffix("_xyz.npy"), buf_xyz)
                cv2.imwrite(str(base.with_suffix("_amp.png")),  amp_u8)
                cv2.imwrite(str(base.with_suffix("_dist.png")), buf_dist)
                cv2.imwrite(str(base.with_suffix("_conf.png")), buf_conf)
    
            return T_cam_tag
    
        T = await loop.run_in_executor(None, _post_process)
        return T, ts_epoch
        



async def main():
    # Load IP, PORT configuration from config
    config = load_config()
    target_address = config['clients']['camera']['target_address']  
    target_port = config['clients']['camera']['target_port']

    

    

    
    # Create socket
    UDPClientSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
    server_address =  (target_address, target_port)

    # CREATE CameraManager object
    cm = CameraManager(cfg = config['camera_manager'])
    cm.load_model()

    

    # PREPARE CHILDREN SUBROCESS POOL
    loop = asyncio.get_running_loop()
    executor = ThreadPoolExecutor()


    





    # PREPARE TO START LOOP
    telegram = create_xml_fast([{'IPOC': '0'}])
    print('start_mess = :', telegram)
    # init dependant telegram variables
    watchDog_out = '0'
    proc = None
    thread = None

    cfg_future: asyncio.Future | None = None     # перед while True # background process handler
    while True:
        ################################################
        # SENDING MESSAGES TO CAMERA SERVER-SUBPROCESS
        ################################################  
        TransmitData = telegram
        SendData(TransmitData, UDPClientSocket, server_address)

        


        ################################################
        # RECEIVING MESSAGES FROM CAMERA SERVER-SUBPROCESS
        ################################################ 
        bytesAddressPair = UDPClientSocket.recvfrom(4096)

        ReceivedMessage = bytesAddressPair[0]
        address = bytesAddressPair[1]
        # clientMsg = "Message from Client:{}".format(ReceivedMessage)
        # clientIP = "Client IP Address:{}".format(address)
        #print(clientMsg)
        #print(clientIP)
        received_dict = extract_xml(ReceivedMessage)
        ##print('camera_received_telegram', ReceivedMessage)

        # use received data from cam
        try:
            watchDog_out = received_dict['WatchDog_out']
        except Exception as ex:
            print(f" Data can not be extracted from received to camera telegram: {ex}")   

        ################################################
        # MAIN OPERATION LOGIC
        ################################################ 
        # RUN HEAVY TASK IN BACKGROUND MODE

        # --- Start a background switch --------------------------------
        if received_dict['need_cam_cal'] == "1" and cm.mode == "inference" and cm.state == "free" and cfg_future is None:
            cfg_future = loop.run_in_executor(None, cm.calib_procedure)
            logging.info("Started calibration")

        # --- Check if the calibration has finished -------------------------
        if cfg_future and cfg_future.done():
            try:
                cfg_future.result()          # propagate exceptions
                logging.info("Camera calibration executed successfully")
            except Exception as exc:
                logging.error("Camera calibration failed: %s", exc)                
            finally:
                cfg_future = None            # ready for the next request
           
      








        ################################################
        # END OF MAIN OPERATION LOGIC
        ################################################ 
        # # what to expect in received_dict from robot
        # sent_mess_list.append({"Sen" : {'Type' : 'ServerToCamera'}})    
        # sent_mess_list.append({"WatchDog_out" : watchDog_out})  
        # sent_mess_list.append({'Frame_assigned': frame_assigned})
        # sent_mess_list.append({'Position_reached': position_reached})
        # sent_mess_list.append({'Need_cam_cal': need_cam_cal})
        # sent_mess_list.append({'Screenshot': screenshot}) 






        



        ################################################
        # PREPARING THE MESSAGE TO BE SENT
        ################################################ 
        sent_mess_list = list() # List of dictionaries. every dictionary will be the element of ElementTree
        # sent_mess_list.append({"Sen" : {'Type' : 'ImFree'}})  
        # try:
        sent_mess_list.append({"Sen" : {'Type' : 'Camera'}})   
        sent_mess_list.append({'XYZ1': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'XYZ2': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'XYZ3': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'XYZ4': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'CAM_CAL_RES': {'X': '0.0', 'Y': '0.0', 'Z': '0.0', 'A': '0.0', 'B': '0.0', 'C': '0.0'}})
        sent_mess_list.append({"WatchDog_in" : watchDog_out}) 
        sent_mess_list.append({"Frame_assign" : '0'}) 
        sent_mess_list.append({"Move_next_pt" : '0'}) 
        sent_mess_list.append({"Cam_cal_in_proc" : '0'}) 
        sent_mess_list.append({"See_4_targets" : '0'}) 

        #cresting a new telegram to send
        telegram = create_xml_fast(sent_mess_list)
        ##print('camera_sending_telegram', telegram)
        
                
        #display.clear_output(wait=True)
        await asyncio.sleep(0.001) # DO NOT CHANGE






    
if __name__ == '__main__':
    await main() # TO USE IN  JUPYTER NITEBOOK ONLY
    #asyncio.run(main()) # TO USE IN SCRIPT, NOT JUPYTER NITEBOOK

        
        

start_mess = : b'<IPOC>0</IPOC>'
waiting for self.state
loading model params
started
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state


CancelledError: 

In [ ]:
parser = argparse.ArgumentParser()
    parser.add_argument("--pcic-port", help="The pcic port from which images should be received", type=int,
                        required=False, default=50010)
#    parser.add_argument("--image", help="The image to received (default: distance)", type=str,
#                        choices=image_choices, required=False, default="amplitude")
    parser.add_argument("--ip", help="IP address of the sensor (default: 192.168.178.69)",
                        type=str, required=False, default="192.168.178.69")
    parser.add_argument("--xmlrpc-port", help="XMLRPC port of the sensor (default: 50010)",
                        type=int, required=False, default=50010)
    parser.add_argument("--index", help="results file index",
                        type=str, required=False, default=0)                    
    args = parser.parse_args()

   # getter = globals()["get_" + args.image]
        
    cam = O3D(ip=args.ip, xmlrpc_port=args.xmlrpc_port)
    fg = FrameGrabber(cam, pcic_port=args.xmlrpc_port)
fg.start(
        [buffer_id.AMPLITUDE_IMAGE, buffer_id.RADIAL_DISTANCE_IMAGE, buffer_id.CONFIDENCE_IMAGE, buffer_id.XYZ]
    )
    title = "O3D Port {}".format(str(args.pcic_port))
    
    index=0
    subindex=0
#    if args.image == "xyz":
#        display_3d(fg, buf, getter, title)
#    else:
#        display_2d(fg, buf, getter, title, args.index)


    shoot=0
    prev_shoot=0
    data = 'Nothing received'
    #  while True:
    #     newestData = None

    #     keepReceiving = True
    #     while keepReceiving:
    #        try:
    #           data, fromAddr = sock.recvfrom(2048)
    #           if data:
    #              newestData = data
    #        except socket.error as why:
    #           if why.args[0] == EWOULDBLOCK:
    #              keepReceiving = False
    #           else:
    #              raise why

    #     if (newestData):
            # code to handle/parse (newestData) here
    index=1
    subindex=0
    
    today = datetime.date.today()  
    datestamp=today.strftime("%Y_%m_%d")
    timestamp=today.strftime("%H_%M_%S")
    ampm=today.strftime("%p")
    print(datestamp)
    os.makedirs('/mnt/tagmii1/'+datestamp+'/'+ampm, exist_ok=True) 
    
    
    path='/mnt/tagmii1/'+datestamp+'/'+ampm+'/'
    isExist = os.path.exists(path+str(index))
    while isExist:
        index=index+1
        path='/mnt/tagmii1/'+datestamp+'/'+ampm+'/'+str(index)
        isExist = os.path.exists(path)
        print("index is there ", index)
    #todaystr = today.isoformat()   
    #os.makedirs(
    folder_count=0
    for folders in os.listdir(path):  # loop over all files
        if os.path.isdir(os.path.join(path, folders)):  # if it's a directory
            folder_count += 1  # increment counter

    
    index=max(folder_count, index)
    print("starting index is ", index)
    os.makedirs('/mnt/tagmii1/'+datestamp+'/'+ampm, exist_ok=True) 
    
    
    global hand_hist
    is_hand_hist_created = False
    idx=1
    s=1
    cam_was_down=0
    
    while True:
        key=None
        ok=0
        if(cam_was_down):
            print("waiting for camera to restart")
            time.sleep(3)
            ip='192.168.178.69'

            response = os.popen(f"ping -c 1 {ip}").read()

            if "1 received" not in response:

                print(f"Camera DOWW {ip} Ping Unsuccessful")
    
                continue


            cam = O3D(ip=args.ip, xmlrpc_port=args.xmlrpc_port)
            fg = FrameGrabber(cam, pcic_port=args.xmlrpc_port)

            fg.start( [buffer_id.AMPLITUDE_IMAGE, buffer_id.RADIAL_DISTANCE_IMAGE, buffer_id.CONFIDENCE_IMAGE, buffer_id.XYZ])
            cam_was_down=0  
        ip='192.168.178.69'

        response = os.popen(f"ping -c 1 {ip}").read()

        if "1 received" not in response:

             print(f"Camera DOWN {ip} Ping Unsuccessful")
             cam_was_down=1   
             continue
        #trigger_mode = cam.to_json()["ifm3d"]["Apps"][0]["TriggerMode"]
        # if trigger_mode == "1":
        #    print("Camera is in Continuous Trigger Mode")
        #elif trigger_mode == "2":
         #   print("Camera is in Software Trigger Mode")
        # Software Trigger the camera
        try:
            fg.sw_trigger()
        except:
            cam_was_down=1
        #print (msg)   
        received_dict = extract_xml(msg, find='all')
 
      
        if( 'Screenshot' in received_dict.keys()):
           
            Shoot=extract_xml(msg, find=['Screenshot'])
            #print("Shoot is ", Shoot)
            shoot=Shoot['Screenshot']
        #           if data:
        #              newestData = data
        #        except socket.error as why:
        #           if why.args[0] == EWOULDBLOCK:
        #              keepReceiving = False
        #           else:
        #              raise why
#        except:
#                    continue
       
       #################
        # Start timer#
        #print("ok to here")
        cam_pos = list()
        start_time = time.time()
        
        ip='192.168.178.69'
        #response = os.popen(f"ping -c 1 {ip}").read()

        #if "1 received" not in response:

         #    print(f"Camera DOWW {ip} Ping Unsuccessful")
    
          #   continue
        #fg.on_error()
        
       

        try:
#            fg.sw_trigger() 
#            fg.start([buffer_id.AMPLITUDE_IMAGE, buffer_id.RADIAL_DISTANCE_IMAGE, buffer_id.CONFIDENCE_IMAGE, buffer_id.XYZ])
            [ok, frame]=fg.wait_for_frame().wait_for(100)
            print("trying to get frame, success:", ok)
        except:
            print('could not connect to camera')
            cam_was_down=1
            continue
        else:
            print("fg status:", fg.is_running())
 #           continue
 #       fg.on_error(continue)
#        [ok, frame]=fg.wait_for_frame().wait_for(5)
        if ok:
            #print("OK!!", frame.timestamps()
            tmestamps=frame.timestamps()
            #continue
#        while not  :
#            continue
        if not ok:
            continue
            #raise RuntimeError("Timeout while waiting for a frame.")
        #frame1 = getter(buf)
        #NORM_AMPLITUDE_IMAGE, buffer_id.RADIAL_DISTANCE_IMAGE, buffer_id.XYZ]  )
         
      ####live script
        
        a=np.uint8(frame.get_buffer(buffer_id.AMPLITUDE_IMAGE))
       # frame1=frame.get_buffer(buffer_id.NORM_AMPLITUDE_IMAGE)
        
        #a=frame.get_buffer(buffer_id.NORM_AMPLITUDE_IMAGE)
        rows = a.shape[1]
        cols = a.shape[0]
#        print(rows)
#        print(cols)
#        print(a.max()) 
        
        for x in range(0, cols - 1):
            for y in range(0, rows -1):
                #print(a[x,y])
                pixel_value=a[x,y]
                if pixel_value>=16200:  ##hard coded normalisation value
                    a[x,y]=0
                if x>=215:
                    a[x,y]=0
#        print(a.max())     
        # Define the desired range
        array=a
        a, b = 0, 255

        # Min and max of the array
        array_min = np.min(array)
        array_max = np.max(array)

        # Normalize to the range [a, b]
        frame1 = a + ( (array - array_min) * (b - a) / (array_max - array_min) )
        #frame2=cv2.normalize(a, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
        frame2=np.uint8(frame.get_buffer(buffer_id.RADIAL_DISTANCE_IMAGE))
        frame3=np.uint8(frame.get_buffer(buffer_id.CONFIDENCE_IMAGE))
#print(frame2.shape)
        pt=frame.get_buffer(buffer_id.XYZ)
#        print("pt", np.max(pt))
        '''
        ###from file script
#        pth=
#        print(pth)
        root="/home/niall/Pictures/listons/2023_12_05/AM/"
        while (len(list(pathlib.Path(root+str(s)).glob("*_i"+str(s)+"_s"+str(idx)+"_amp.png")))==0) and s<1000:
            s=s+1
            #break
        
        pth=list(pathlib.Path(root+str(s)).glob("*_i"+str(s)+"_s"+str(idx)+"_amp.png"))[0]
            #idx=1i doesnt reset need to fox i recording program
        frame1=np.uint8(Image.open(pth))
        frame2=np.uint8(Image.open(str(pth).replace("amp", "dist")))
        frame3=np.uint8(Image.open(str(pth).replace("amp", "conf")))
        pt=np.load(str(pth).replace("amp.png", "pcd2.npy"))
        np.set_printoptions(threshold=sys.maxsize)
#        print(pt.shape)
#        print(pt)
        pt = pt.reshape((176,132,3))
        
        idx=idx+1
#        print(pt.shape)
#        print(pt)
        '''
        
        
        
         ###savevscript
        
        if(int(shoot) != prev_shoot  ):
            prev_shoot=int(shoot)
            if(int(shoot) == 1 ):
                index=index+1
                subindex=0
                print ("Index is", index)
                today = datetime.date.today()  
                datestamp=today.strftime("%Y_%m_%d")
                timestamp=today.strftime("%Y_%m_%d_%H_%M_%S")
                #todaystr = today.isoformat()   
                #os.makedirs(
                os.makedirs('/mnt/tagmii1/'+datestamp+'/'+ampm+'/'+str(index), exist_ok=True)
            
       
        #filestr="/media/niall/tagmii1/" + str(idx)
        filestr='/mnt/tagmii1/'+datestamp+'/'+ampm+'/'+str(index)+'/'+timestamp+'_i'+str(index)+'_s'+str(subindex)
  
  #uncooment here to save images   
     
#        im1 = Image.fromarray(frame1)
#            
#        im1.save(filestr+'_amp.png')
#        im2 = Image.fromarray(frame2)
#        im2.save(filestr+'_dist.png')
#        im3 = Image.fromarray(frame3)
#        im3.save(filestr+'_conf.png')
#        ptw = pt.reshape(pt.shape[:-3] + (-1, 3))
#        np.save(filestr+'_pcd2', ptw)
        idx=idx+1
#        frame1=cv2.normalize(frame.get_buffer(buffer_id.NORM_AMPLITUDE_IMAGE), None, 0,256, cv2.NORM_MINMAX, cv2.CV_8U)
#       # frame1=frame.get_buffer(buffer_id.NORM_AMPLITUDE_IMAGE)
#        frame2=cv2.normalize(frame.get_buffer(buffer_id.RADIAL_DISTANCE_IMAGE), None, 0,256, cv2.NORM_MINMAX, cv2.CV_8U)
#       frame3=cv2.normalize(frame.get_buffer(buffer_id.CONFIDENCE_IMAGE), None, 0,256, cv2.NORM_MINMAX, cv2.CV_8U)
#        pt=frame.get_buffer(buffer_id.XYZ)
        # Adjusts the brightness by adding 10 to each pixel value 
        brightness = -0 
        # Adjusts the contrast by scaling the pixel values by 2.3 
        contrast = 0.6
       # frame1 = cv2.addWeighted(frame1, contrast, np.zeros(frame1.shape, frame1.dtype), 0, brightness) 
        #frame1 = cv2.addWeighted(frame2, 0.6, np.zeros(frame2.shape, frame2.dtype), 0, 0) 
        frame1=frame2*0.8
        #frame3 = cv2.addWeighted(frame3, -1.0, np.zeros(frame3.shape, frame3.dtype), 0, 60)   
        
        
        c = frame3# np.array([1, 4, 2, 5, 7, 4, 2, 5, 6, 7, 7, 2, 5])
        mapping = np.arange(c.max() + 1)
        map_from = np.array([45, 48, 56,  61])
        map_to = np.array([15, 60, 180, 180])
        mapping[map_from] = map_to
        frame3=np.uint8(mapping[c])
        #pressed_key = cv2.waitKey(1)
        #cv2.imshow("teat_detector", frame1)

- [ ] Logic for Extract transformation mtx from camera

#See "4data_recording_ifm3d_autocal_listons_append.py" attached  lines 507-567

#This saves text file with corners and transformation matrix from apriltag code

#I then run a code based on ros transformations to parse this file using regex. This caused so many headaches! I recommend saving to json/xml instead
#I will share the ros code in a separate email. When you get to that stage, let me know, I don't  want to confuse things for now.


#Sorry for the delay and the messy code :)

In [ ]:
# SYNC VERSION

from IPython import display

import time
from lib_xml_tree import *
from lib_connection import *
from lib_config_loader import load_config


    
if __name__ == '__main__':

    # Load IP, PORT configuration from config
    config = load_config()
    target_address = config['clients']['camera']['target_address']  
    target_port = config['clients']['camera']['target_port']

    UDPClientSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
    server_address =  (target_address, target_port)

    
    telegram = create_xml_fast([{'IPOC': '0'}])
    print('start_mess = :', telegram)

    # init dependant telegram variables
    watchDog_out = '0'
    
    while True:
        ################################################
        # SENDING MESSAGES TO CAMERA SERVER-SUBPROCESS
        ################################################  
        TransmitData = telegram
        SendData(TransmitData, UDPClientSocket, server_address)

        


        ################################################
        # RECEIVING MESSAGES FROM CAMERA SERVER-SUBPROCESS
        ################################################ 
        bytesAddressPair = UDPClientSocket.recvfrom(4096)

        ReceivedMessage = bytesAddressPair[0]
        address = bytesAddressPair[1]
        # clientMsg = "Message from Client:{}".format(ReceivedMessage)
        # clientIP = "Client IP Address:{}".format(address)
        #print(clientMsg)
        #print(clientIP)
        received_dict = extract_xml(ReceivedMessage)
        print('camera_received_telegram', ReceivedMessage)

                # use received data from cam
        try:
            watchDog_out = received_dict['WatchDog_out']
        except Exception as ex:
            print(f" Data can not be extracted from received to camera telegram: {ex}")   





        ################################################
        # PREPARING THE MESSAGE TO BE SENT
        ################################################ 
        sent_mess_list = list() # List of dictionaries. every dictionary will be the element of ElementTree
        # sent_mess_list.append({"Sen" : {'Type' : 'ImFree'}})  
        # try:
        sent_mess_list.append({"Sen" : {'Type' : 'Camera'}})   
        sent_mess_list.append({'XYZ1': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'XYZ2': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'XYZ3': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'XYZ4': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'CAM_CAL_RES': {'X': '0.0', 'Y': '0.0', 'Z': '0.0', 'A': '0.0', 'B': '0.0', 'C': '0.0'}})
        sent_mess_list.append({"WatchDog_in" : watchDog_out}) 
        sent_mess_list.append({"Frame_assign" : '0'}) 
        sent_mess_list.append({"Move_next_pt" : '0'}) 
        sent_mess_list.append({"Cam_cal_in_proc" : '0'}) 
        sent_mess_list.append({"See_4_targets" : '0'}) 

        #cresting a new telegram to send
        telegram = create_xml_fast(sent_mess_list)
        print('camera_sending_telegram', telegram)
        
                
        display.clear_output(wait=True)


        


        
        

In [1]:
import socket
import io
import xml.etree.ElementTree as xml

In [1]:
#for test only 
# Creates structure for ambedding into the xml package, which will be send to robot
def create_Rkorr_for_sending(frame = {'X' : 0, 'Y' : 0, 'Z' : 0}):  #frame = {'X' : float, 'Y' : float, 'Z' : float, 'A' : float, 'B' : float, 'C' : float}
    # print(frame)
    Rkorr = {'X' : str(frame['X']), 
             'Y' : str(frame['Y']), 
             'Z' : str(frame['Z'])}
    
    return Rkorr


def create_xml(target_frame_1 = {'X' : 0.100, 'Y' : 1.000, 'Z' : 0.030, 'A' : 0.000, 'B' : 0.000, 'C' : 0.000},
              target_frame_2 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              target_frame_3 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              target_frame_4 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              IPOCtimerReseived = '123456789'):

    SenAttr = dict(Type='ImFree')

    root = xml.Element("Sen", SenAttr)
    
    ################# RESTORE IT ##################
    RkorrArrt_1 = create_Rkorr_for_sending(target_frame_1)
    Rkorr_1 = xml.SubElement(root, "RKorr", RkorrArrt_1)
    ################################################
    #print(RkorrArrt_1)
    #print(Rkorr_1)





    #RkorrArrt = dict(X='0.03')
    #Rkorr = xml.SubElement(root, "RKorr", RkorrArrt)
    #print(RkorrArrt) 
   




    
#     RkorrArrt_2 = create_Rkorr_for_sending(target_frame_2)
#     Rkorr_2 = xml.SubElement(root, "Rkorr", RkorrArrt_2)
    
#     RkorrArrt_3 = create_Rkorr_for_sending(target_frame_3)
#     Rkorr_3 = xml.SubElement(root, "Rkorr", RkorrArrt_3)
    
#     RkorrArrt_4 = create_Rkorr_for_sending(target_frame_4)
#     Rkorr_4 = xml.SubElement(root, "Rkorr", RkorrArrt_4)
    
    
    IPOC = xml.SubElement(root, "IPOC")
    IPOC.text = IPOCtimerReseived
    # IPOC.text = '123456789'
    
    
    tree = xml.ElementTree(root)
    
    
    DecodedTree = xml.tostring(root, encoding='utf8', method='xml', short_empty_elements='true', xml_declaration=False)
    return(DecodedTree)



In [3]:
xml_parcel = create_xml()
xml_parcel

b'<Sen Type="ImFree"><RKorr X="0.1" Y="1.0" Z="0.03" /><IPOC>123456789</IPOC></Sen>'

In [ ]:
d = {'XYZ1' : '123'}
